# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Importing Python packages 
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# get the current working directory
#print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
    
# Gathering the paths of the csv files
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*.csv'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below to see what the list of event data rows will look like
#print(full_data_rows_list)

In [4]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        #if artist name is empty, skip that row.
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
f.close()

6821


# Part II. Inserting Data Into Apache Cassandra

## The file <font color=red>event_datafile_new.csv</font> will be used to create tables based on queries. The columns of the data is as follows:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
# Creating a Cassandra instance in local machine 
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating a Keyspace

In [7]:
# A keyspace in Cassandra is a namespace that defines data replication on nodes. A cluster contains one keyspace per node.
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS csndra 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting the Keyspace

In [8]:
try:
    session.set_keyspace('csndra')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
# file name
file = 'event_datafile_new.csv'
# column names of the data in 'event_datafile_new.csv'
header_list = ['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId']

### QUERY 1

In [10]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "SELECT * FROM music_app_session WHERE session_id = 338 AND item_in_session = 4"                

In [11]:
# CREATING TABLE FOR QUERY 1
query = "CREATE TABLE IF NOT EXISTS music_app_session "
# One unique session_id could have many items in session. Together they form a unique pair.
# The primary key has been defined by focusing on the WHERE statement in our query.
query = query + "(artist_name varchar, song_title varchar, song_length double, session_id int, item_in_session int, \
PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Opening file to read
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_session (artist_name, song_title, song_length, session_id, item_in_session)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        
        # matching header for that particular row in the loop as key:value pairs - a dict to increase readibility.
        match_row = dict(zip(header_list,line))
        
        try:
            session.execute(query, (match_row['artist'], match_row['song'], float(match_row['length']), \
                                int(match_row['sessionId']), int(match_row['itemInSession'])))
        except Exception as e:
            print(e)
f.close()

In [13]:
## Verifying the data was entered into the table
## Since the where condition (session_id and item_in_session) represents the primary key in the table,
## the result is a single row of data.
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, ',', row.song_title, ',', row.song_length)

Faithless , Music Matters (Mark Knight Dub) , 495.3073


### QUERY 2

In [14]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "SELECT * FROM music_app_artist_user WHERE user_id = 10 AND session_id = 182"                    

In [15]:
# CREATING TABLE FOR QUERY 2
query = "CREATE TABLE IF NOT EXISTS music_app_artist_user "
# partition key: user_id, clustering columns: item_in_session for sorting, session_id to make the PK unique, because
# there can be duplicate user_id and item_in_session in different session_ids
query = query + "(artist_name varchar, song_title varchar, first_name varchar, last_name varchar, item_in_session int, \
user_id int, session_id int, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# file name
file = 'event_datafile_new.csv'
# column names of the data in 'event_datafile_new.csv'
header_list = ['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId']
# opening file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_artist_user (artist_name, song_title, first_name, last_name, item_in_session, \
        user_id, session_id)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        # matching header for that particular row in the loop as key:value pairs - a dict to increase readibility.
        match_row = dict(zip(header_list,line))
        
        try:
            session.execute(query, (match_row['artist'], match_row['song'], match_row['firstName'], match_row['lastName'], \
                                    int(match_row['itemInSession']), int(match_row['userId']), int(match_row['sessionId'])))
        except Exception as e:
            print(e)
f.close()

In [17]:
## Verifying the data was entered into the table
## For user_id 10 in session_id 182, there are 4 corresponding songs existing.

# One session_id belongs to one particular user_id; therefore,
# it's expected to get the same user names in the output.

### The output is ordered based on the order of clustering columns, meaning 
### it's ordered firstly by session_id, then item_in_session. 
### Since session_id is fixed at 182, the output is ordered by item_in_session
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, ',', row.song_title, ',', row.first_name, ',', row.last_name)

Down To The Bone , Keep On Keepin' On , Sylvie , Cruz
Three Drives , Greece 2000 , Sylvie , Cruz
Sebastien Tellier , Kilometer , Sylvie , Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie , Cruz


### QUERY 3

In [18]:
## Query 3: Give me every user name (first and last) in my music app history
## who listened to the song 'All Hands Against His Own'
query3 = "SELECT * FROM music_app_song_user WHERE song_title = 'All Hands Against His Own'" 

In [19]:
# CREATING TABLE FOR QUERY 3
query = "CREATE TABLE IF NOT EXISTS music_app_song_user "
# partition key: song_title, clustering columns: user_id and session_id to make the PK unique, because
# there can be duplicate song_title and user_id together in different sessions.
query = query + "(first_name varchar, last_name varchar, song_title varchar, user_id int, session_id int, \
PRIMARY KEY (song_title, user_id, session_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
# opening file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_song_user (first_name, last_name, song_title, user_id, session_id)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        
        # matching header for that particular row in the loop as key:value pairs - a dict to increase readibility.
        match_row = dict(zip(header_list,line))
        
        try:
            session.execute(query, (match_row['firstName'], match_row['lastName'], match_row['song'], int(match_row['userId']),\
                                    int(match_row['sessionId'])))
        except Exception as e:
            print(e)
f.close()

In [21]:
## Verifying the data was entered into the table

## There are three users who listened to 'All Hands Against His Own'
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, ',', row.last_name)

Jacqueline , Lynch
Tegan , Levine
Sara , Johnson


### Dropping the tables before closing out the sessions

In [22]:
# Dropping tables before closing out the sessions
query1_drop = "drop table music_app_session"
try:
    rows = session.execute(query1_drop)
except Exception as e:
    print(e)
    
query2_drop = "drop table music_app_artist_user"
try:
    rows = session.execute(query2_drop)
except Exception as e:
    print(e)
    
query3_drop = "drop table music_app_song_user"
try:
    rows = session.execute(query3_drop)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()